#**El esquema de una BD**


*   Especifica la estructura lógica de las tablas de una BD (nombres de las tablas y de cada uno de sus atributos) y cómo estas se relacionan.
*   Se define al momento de crear una BD (en general no cambia). También se debe asociar un tipo de dato a cada atributo.
*   Tiene una representación gráfica que es muy útil al momento de escribir consultas SQL (en la jerga se habla de `queries`).

A continuación tenemos el esquema de la BD Chinook, una BD de una tienda de venta de música que utilizaremos de aquí en adelante:

<center><img src="https://drive.google.com/uc?id=1fKhtlmyFD2Vryk3ubFX4g6FsS0PNLNaZ" alt="drawing" width=700px></center>

#**La instancia de una BD**


*   Corresponde a los datos concretos almacenados en las tablas. Por ejemplo, los datos de las facturas.
*   Puede cambiar pero respetando lo indicado en el esquema.
*   A continuación una muestra de los datos de facturas almacenados en la BD Chinook.

<center><img src="https://drive.google.com/uc?id=1tuOpSVxsTtmhdj77NMde7D0IJZKlMl2C"></center>

#**Clave primaria y clave foránea.**


*   Una clave primaria (primary key PK) es un atributo que permite identificar de manera única a los registros de una tabla. <br>
    **IMPORTANTE**: Una clave primaria no admite valores nulos.  
*   Las tablas se relacionan por medio de columnas en común. Esto permite representar la relación entre registros de diferentes tablas.
*   Para materializar dicha relación el uso de una clave foránea (foreign key FK) es fundamental.   
*   La clave foránea de una tabla es una clave primaria en otra. De esta manera una fila de una tabla se relaciona con una o más fila de otra.

A continuación veremos la relación entre los datos de clientes (tabla `Customer`) y sus facturas (tabla `Invoice`) almacenados en la BD Chinook:

<center><img src="https://drive.google.com/uc?id=1hCsroFJ9YCRPIV33O-mifIjXicdFi5CW"></center>

#**Operadores condicionales**

Usualmente, todas las consultas SQL (a excepción de la sentencia `INSERT`) pueden incluir una cláusula condicional, la cual nos permite filtrar los registros de una tabla por ciertas condiciones. Para esto nosotros tenemos que utilizar la sentencia `WHERE`.

```sql
SELECT columna1 FROM tabla WHERE condicion
```

donde `condicion` puede incluir cualquiera de los operadores condicionales:

- \> (Mayor), >= (Mayor o igual)
- < (Menor), <= (Menor o igual)
- = (Igual), != (No igual), <> (No igual)
- NOT (Negación)
- IN (especificar múltiples valores)
- BETWEEN (Especificar un rango de valores).

Por otra parte, podemos añadir múltiples condiciones utilizando los operadores `OR`, y el operador `AND`. Entonces, reescribiendo la consulta anterior, a modo de ejemplo tenemos:

```sql
SELECT * FROM tabla WHERE columna1 > 50;
SELECT columna1 FROM tabla WHERE columna2 > 50;
SELECT columna1, columna2 FROM tabla WHERE (columna2 < 10 AND columna1 > 5) OR columnaN = 1;
```
Analizaremos algunos ejemplos utilizando la base de datos de ejemplo Chinook.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sqlite3 #Importamos la biblioteca

#Generamos la conexion
conn = sqlite3.connect("/content/drive/MyDrive/06_SQL/Chinook_Sqlite.sqlite") #En este archivo está la BD Chinook.
cursor = conn.cursor() #Generamos nuestro cursor para las consultas, recuerde que también podemos usar pandas.

Veremos los nombres de las tablas de la BD consultando el catálogo de SQLite:

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';") #Seleccionamos la columna Nombre de la tabla sqlite_master, donde la columna type sea igual a 'table'
print(cursor.fetchall()) #Le pedimos a nuestro cursor que obtenga todos los resultados y los imprima:

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',), ('Track_BKP',), ('TrackLog',)]


In [ ]:
import pandas as pd

In [ ]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';",conn)

,name
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


Podemos ver entonces que existen 11 tablas en nuestra base de datos, cada una relacionada de distinta manera. Supongamos que queremos ver cuáles son las columnas que existen en una de estas tablas, donde nosotros utilizaremos la sentencia `PRAGMA`. Si bien, esta sentencia corresponde a operaciones más avanzadas, nosotros la podremos definir como: Una extensión de SQL para SQLite, la cual nos permite modificar la operación de SQLite, o en su defecto, consultar por datos internos de una base de datos.

Específicamente, nosotros utilizaremos el método implementado en `PRAGMA` llamado `table_info`, el cual permitirá conocer las distintas columnas pertenecientes a una tabla. Esta consulta retornará información de las columnas en el siguiente orden:

- Id de la columna
- Nombre de la columna
- Tipo de dato asociada a la columna
- Flag indicando si la columna puede ser nula o no
- Valor por defecto para dicha columna
- Flag de clave primaria


Mayor información la pueden revisar en la [documentación de PRAGMA](https://www.sqlite.org/pragma.html)


In [ ]:
cursor.execute("PRAGMA table_info('Invoice')")
for el in cursor.fetchall():
    print(el)

(0, 'InvoiceId', 'INTEGER', 1, None, 1)
(1, 'CustomerId', 'INTEGER', 1, None, 0)
(2, 'InvoiceDate', 'DATETIME', 1, None, 0)
(3, 'BillingAddress', 'NVARCHAR(70)', 0, None, 0)
(4, 'BillingCity', 'NVARCHAR(40)', 0, None, 0)
(5, 'BillingState', 'NVARCHAR(40)', 0, None, 0)
(6, 'BillingCountry', 'NVARCHAR(40)', 0, None, 0)
(7, 'BillingPostalCode', 'NVARCHAR(10)', 0, None, 0)
(8, 'Total', 'NUMERIC(10,2)', 1, None, 0)


In [ ]:
pd.read_sql("PRAGMA table_info('Invoice')",conn)

,cid,name,type,notnull,dflt_value,pk
0,0,InvoiceId,INTEGER,1,None,1
1,1,CustomerId,INTEGER,1,None,0
2,2,InvoiceDate,DATETIME,1,None,0
3,3,BillingAddress,NVARCHAR(70),0,None,0
4,4,BillingCity,NVARCHAR(40),0,None,0
5,5,BillingState,NVARCHAR(40),0,None,0
6,6,BillingCountry,NVARCHAR(40),0,None,0
7,7,BillingPostalCode,NVARCHAR(10),0,None,0
8,8,Total,"NUMERIC(10,2)",1,None,0


Estos comandos son útiles cuando no tenemos una representación gráfica de una base de datos. En este caso, vemos que la tabla `Invoice`, está compuesta por 9 columnas.

##**Con todos estos antecedentes, procedamos a hacer consultas a la base de datos.**

Primero, veamos la tabla `Invoice`, seleccionemos los primeros 5 elementos utilizando el método `cursor.fetchmany(N)` donde `N = 5`

In [ ]:
cursor.execute("SELECT * FROM Invoice")
registers = cursor.fetchmany(5)
for reg in registers:
    print(reg)

(1, 2, '2009-01-01 00:00:00', 'Theodor-Heuss-Straße 34', 'Stuttgart', None, 'Germany', '70174', 1.98)
(2, 4, '2009-01-02 00:00:00', 'Ullevålsveien 14', 'Oslo', None, 'Norway', '0171', 3.96)
(3, 8, '2009-01-03 00:00:00', 'Grétrystraat 63', 'Brussels', None, 'Belgium', '1000', 5.94)
(4, 14, '2009-01-06 00:00:00', '8210 111 ST NW', 'Edmonton', 'AB', 'Canada', 'T6G 2C7', 8.91)
(5, 23, '2009-01-11 00:00:00', '69 Salem Street', 'Boston', 'MA', 'USA', '2113', 13.86)


In [ ]:
pd.read_sql("SELECT * FROM Invoice",conn).head(5)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


Ahora, consultemos todos los datos de los clientes (almacenados en la tabla `Customer`):

In [ ]:
pd.read_sql("SELECT * FROM Customer",conn)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
5,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
6,7,Astrid,Gruber,None,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,+43 01 5134505,None,astrid.gruber@apple.at,5
7,8,Daan,Peeters,None,Grétrystraat 63,Brussels,None,Belgium,1000,+32 02 219 03 03,None,daan_peeters@apple.be,4
8,9,Kara,Nielsen,None,Sønder Boulevard 51,Copenhagen,None,Denmark,1720,+453 3331 9991,None,kara.nielsen@jubii.dk,4
9,10,Eduardo,Martins,Woodstock Discos,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,+55 (11) 3033-4564,eduardo@woodstock.com.br,4


Cabe destacar que podríamos realizar la siguiente operación:

```python
cursor.execute("SELECT * FROM Invoice")
registers = cursor.fetchall()
```

el problema que podría surgir acá, es que si tenemos una gran cantidad de registros puede causar que nuestra memoria RAM se llene. Esto traería una serie de consecuencias importantes en nuestra máquina. Por esto mismo, filtrar por condiciones lógicas puede ser una buena opción. En este caso, la cantidad de registros existentes en la tabla Invoice, no son una cantidad critica, por lo que podemos ejecutar sin ningún problema.


In [ ]:
cursor.execute("SELECT * FROM Invoice") #Ejecutamos la sentencia
all_registers = cursor.fetchall() #obtenemos todos los registros que cumplan la sentencia
print(len(all_registers)) #Imprimimos la cantiodad de elementos que existen

412


Tenemos 412 registros, los cuales podríamos imprimir por pantalla, pero para legibilidad del notebook vamos a omitir esa parte. La consulta ahora que podemos hacer:

**¿Cuantos registros existen en la tabla Invoice con un valor de factura mayor a 20?**

In [ ]:
df = pd.read_sql("""
                  SELECT *
                  FROM Invoice
                  WHERE Total > 20
                  """,conn)
df

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,96,45,2010-02-18 00:00:00,Erzsébet krt. 58.,Budapest,None,Hungary,H-1073,21.86
1,194,46,2011-04-28 00:00:00,3 Chatham Street,Dublin,Dublin,Ireland,None,21.86
2,299,26,2012-08-05 00:00:00,2211 W Berry Street,Fort Worth,TX,USA,76110,23.86
3,404,6,2013-11-13 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,25.86


In [ ]:
print("La cantidad de registros con un valor de factura mayor a 20 es:",len(df))

La cantidad de registros con un valor de factura mayor a 20 es: 4


**¿Cuantos registros existen en la tabla Invoice con un valor de factura menor a 10?**


In [ ]:
df2 = pd.read_sql("""
                  SELECT *
                  FROM Invoice
                  WHERE Total < 10
                  """,conn)
df2

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,6,37,2009-01-19 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,0.99
...,...,...,...,...,...,...,...,...,...
343,407,23,2013-12-04 00:00:00,69 Salem Street,Boston,MA,USA,2113,1.98
344,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96
345,409,29,2013-12-06 00:00:00,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
346,410,35,2013-12-09 00:00:00,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91


In [ ]:
print("La cantidad de registros con un valor de factura menor a 10 es:", len(df2))

La cantidad de registros con un valor de factura menor a 10 es: 348


A su vez, podemos consultar por valores específicos utilizando el operador `=`, o algún otro operador relacional/lógico. En este caso podemos hacer una pregunta a la tabla `Track`, donde preguntaremos:

**¿Cuantas canciones pertenecen al Género Musical con identificador igual a 1?**

**¿Cuantas canciones pertenecen al Género Musical con identificador distinto de 1?**


Primero exploremos el contenido de la tabla **Genre**:

In [ ]:
pd.read_sql("SELECT * FROM Genre",conn)

,GenreId,Name
0,1,Rock
1,2,Jazz
2,3,Metal
3,4,Alternative & Punk
4,5,Rock And Roll
5,6,Blues
6,7,Latin
7,8,Reggae
8,9,Pop
9,10,Soundtrack


**¿Cuantas canciones pertenecen al Género Musical con identificador igual a 1?**


In [ ]:
df3 = pd.read_sql("""
                  SELECT *
                  FROM Track
                  WHERE GenreId = 1
                  """,conn)
print("La cantidad de canciones con el identificador de Genero Musical igual a 1 es:",len(df3))

La cantidad de canciones con el identificador de Genero Musical igual a 1 es: 1297


**¿Cuantas canciones pertenecen al Género Musical con identificador distinto de 1?**

In [ ]:
df4 = pd.read_sql("""
                  SELECT *
                  FROM Track
                  WHERE GenreId != 1
                  """,conn)
print("La cantidad de canciones con el identificador de Genero Musical distinto a 1 son:",len(df4))

La cantidad de canciones con el identificador de Genero Musical distinto a 1 son: 2206


Alternativamente, para la última consulta podemos utilizar el operador `NOT` sobre la condición lógica de igualdad y obtener el mismo resultado.

```SQL
SELECT Columna1,Columna2,...,ColumnaN FROM Tabla WHERE NOT(condición lógica)
```

y la consulta anterior queda de la siguiente manera:


In [ ]:
df4 = pd.read_sql("""
                  SELECT *
                  FROM Track
                  WHERE NOT (GenreId = 1)
                  """,conn)
print("Alternativamente, utilizando el operador NOT",len(df4)) # Vamos a ver que las cantidades coinciden


Alternativamente, utilizando el operador NOT 2206


Para los operadores `IN` y `BETWEEN` la sintaxis puede ser distinta a una consulta típica de SQL, donde estas son:
```SQL
SELECT * FROM Tabla WHERE Columna IN (Val1,Val2,...,ValN) -- Uso del IN
SELECT * FROM Tabla WHERE Columna BETWEEN Val1 AND Val2 -- Uso del BETWEEN
```

llevando esto a nuestros ejemplos, podemos hacer las consultas:

**¿Cuantas canciones existen las cuales pertenezcan a los géneros musicales definidos por los identificadores 1, 2 o 3?**

In [ ]:
len(pd.read_sql("SELECT * FROM Track WHERE GenreId IN (1,2,3)",conn))

1801

Hasta el momento para calcular la cantidad de registros hemos utilizado la función de python `len`. Sin embargo, SQL también tiene una función para ello, la función `COUNT`:

In [ ]:
pd.read_sql("""
            SELECT COUNT(*)
            FROM Track
            WHERE GenreId IN (1,2,3)
            """,conn)

,COUNT(*)
0,1801


**¿Cuantas canciones existen las cuales tengan una duración entre 100000 y 300000 Milisegundos?**

In [ ]:
pd.read_sql("""
            SELECT COUNT(*)
            FROM Track
            WHERE Milliseconds BETWEEN 100000 AND 300000
            """,conn)

,COUNT(*)
0,2376


Alternativamente, para a las sentencias `IN` y `BETWEEN` podemos reemplazarlas con sentencias con múltiples condiciones, las cuales serían equivalentes a los usos anteriores:

In [ ]:
len(pd.read_sql("""
                SELECT *
                FROM Track
                WHERE GenreId =  1 OR GenreId =  2 OR GenreId =  3
                """,conn))

1801

In [ ]:
len(pd.read_sql("""
                SELECT *
                FROM Track
                WHERE Milliseconds >= 100000 AND Milliseconds <= 300000
                """,conn))

2376

**¿Podemos utilizar el operador NOT en conjunto con el operador BETWEEN?** La respuesta es sí:

In [ ]:
len(pd.read_sql("""
                SELECT *
                FROM Track
                WHERE Milliseconds NOT BETWEEN 100000 AND 300000
                """,conn))

1127

y si queremos saber las canciones que en su nombre tienen la palabra **love**?.Veamos la estructura de la tabla `Track`:

In [ ]:
pd.read_sql("PRAGMA table_info('Track')",conn)

,cid,name,type,notnull,dflt_value,pk
0,0,TrackId,INTEGER,1,None,1
1,1,Name,NVARCHAR(200),1,None,0
2,2,AlbumId,INTEGER,0,None,0
3,3,MediaTypeId,INTEGER,1,None,0
4,4,GenreId,INTEGER,0,None,0
5,5,Composer,NVARCHAR(220),0,None,0
6,6,Milliseconds,INTEGER,1,None,0
7,7,Bytes,INTEGER,0,None,0
8,8,UnitPrice,"NUMERIC(10,2)",1,None,0


y ahora escribamos la consulta:

In [ ]:
pd.read_sql("""
            SELECT Name
            FROM Track
            WHERE Name like '%love%'
            """,conn)

,Name
0,Love In An Elevator
1,"Love, Hate, Love"
2,Let Me Love You Baby
3,My Love
4,The Girl I Love She Got Long Black Wavy Hair
...,...
109,Love Comes
110,Arms Around Your Love
111,Love Is a Losing Game
112,I Heard Love Is Blind


#**Funciones de agregación**

Las funciones de agregación realizan una operación específica sobre todas las filas en una tabla. Estas funciones usualmente devuelven un valor resumiendo los datos de las columnas que nosotros consultemos. Las funciones de agregación implementadas en cada motor de bases de datos pueden variar, pero las 5 más comunes (y que están implementadas en `SQLite`) son:

- Min: Valor Mínimo
- Max:  Valor Máximo
- Avg: Media (Promedio)
- Sum: Suma total de los valores
- Count: Cantidad de elementos **No Nulos**

La sintaxis para utilizar estas funciones corresponde a la siguiente:


```SQL
SELECT FUNC(column) FROM Tabla -- Donde FUNC, puede ser una de las 5 declaradas funciones de agregación declaradas anteriormente
```

A modo de ejemplo utilizando la tabla `Track`, podemos ver los valores mínimos y máximos para la duración de las pistas musicales:


In [ ]:
#Obtenemos el valor minimo de los milisegundos
pd.read_sql("SELECT MIN(Milliseconds) FROM Track",conn)

,MIN(Milliseconds)
0,1071


In [ ]:
#Obtenermos el valor maximo
pd.read_sql("SELECT MAX(Milliseconds) FROM Track",conn)

,MAX(Milliseconds)
0,5286953


In [ ]:
# También podemos consultar por uno o más valores
pd.read_sql("SELECT MIN(Milliseconds), MAX(Milliseconds) FROM Track",conn)

,MIN(Milliseconds),MAX(Milliseconds)
0,1071,5286953


Para calcular la cantidad de facturas cuyo valor es mayor que 20 utiizaremos la función **COUNT**:

In [ ]:
pd.read_sql("""
            SELECT COUNT(Total)
            FROM Invoice
            WHERE Total > 20
            """,conn)

,COUNT(Total)
0,4


Adicionalmente, podemos hacer operaciones matemáticas sobre las funciones de agregación (de la misma forma que se podría hacer sobre una columna con valores numéricos). En este caso, transformaremos los milisegundos a minutos.

In [ ]:
pd.read_sql("""
              SELECT COUNT(Milliseconds), AVG(Milliseconds) / (60*1000),  SUM(Milliseconds) / (60*1000)
              FROM Track
            """,conn)

,COUNT(Milliseconds),AVG(Milliseconds) / (60*1000),SUM(Milliseconds) / (60*1000)
0,3503,6.559987,22979


Finalmente, podemos agregar la cláusula `WHERE` para poder filtrar filas según criterios, y luego sobre estos registros filtrados, realizar la agregación:

In [ ]:
pd.read_sql("""
              SELECT COUNT(Milliseconds), AVG(Milliseconds) / (60*1000),  SUM(Milliseconds) / (60*1000)
              FROM Track
              WHERE GenreId = 1
            """,conn)

,COUNT(Milliseconds),AVG(Milliseconds) / (60*1000),SUM(Milliseconds) / (60*1000)
0,1297,4.731834,6137


**Nota:** Cabe destacar que incluir columnas **no agregadas** en una consulta donde exista por lo menos una **función de agregación**, puede que devuelva como respuesta un error (dependiendo del motor de base de datos, ej: mysql). En el caso de SQLite, debería devolver el valor de la columna la cual se asocia el valor agregado (para las funciones MIN y MAX), o el último valor (para otras funciones de agregación). Otros comportamientos distintos: **[Quirks de SQLite](https://sqlite.org/quirks.html)**

Ejemplo de este comportamiento:


In [ ]:
pd.read_sql("SELECT Trackid, MAX(Milliseconds) FROM Track",conn)

,TrackId,MAX(Milliseconds)
0,2820,5286953


In [ ]:
pd.read_sql("SELECT Trackid, MIN(Milliseconds) FROM Track",conn)

,TrackId,MIN(Milliseconds)
0,2461,1071


In [ ]:
pd.read_sql("SELECT Trackid, AVG(Milliseconds) FROM Track",conn)

,TrackId,AVG(Milliseconds)
0,1,393599.212104


In [ ]:
pd.read_sql("SELECT Trackid, SUM(Milliseconds) FROM Track",conn)

,TrackId,SUM(Milliseconds)
0,1,1378778040


In [ ]:
pd.read_sql("SELECT MAX(Trackid) FROM Track",conn)

,MAX(Trackid)
0,3503


Con lo anterior verificamos que el valor de Trackid asociado a las funciones AVG y SUM es el valor más grande de la columna Trackid.

#**Actividad 1**

Vamos a hacer exploración de una base de datos de Kaggle, específicamente una base de datos de Fútbol (Soccer), para distintas ligas europeas. Existen múltiples tablas y columnas asociadas a estas, por lo que ustedes tendrán que hacer una exploración de lo que existe en la base de datos. <br>
**IMPORTANTE:** Extraer la BD desde el archivo `European_soccer.zip`.

Fuente: [European Soccer Database](https://www.kaggle.com/hugomathien/soccer)

1. Obtener los nombres de las tablas y las columnas para cada una de estas. **Para discutir:** ¿Las nombres de las columnas son auto explicativas?
2. Muestre los países existentes en la tabla Country, junto a su identificador.
3. ¿Cuantos partidos hay en la tabla Matches para los países **Italia, Suiza y España**? Muestre los primeros 20 registros de este match especificando las primeras 11 columnas de dicha tabla
4. ¿Cuantos partidos hay en la tabla Matches con 5 goles mínimo para el equipo de local o visita?
5. ¿Cuantos registros, valores mínimos, valores máximos, y promedios para las columnas  overall score, shot_power, stamina, short_passing, long_passing? ¿Cómo cambian estos valores si filtramos por un overall_score entre 50 y 70?

6. **Desafío:** ¿Cuantos partidos hay en la tabla Matches con 5 goles mínimo entre los dos equipos?

In [ ]:
#Generamos la conexion
# OJO: adecuar al PATH de tu Drive
conn2 = sqlite3.connect("/content/drive/MyDrive/06_SQL/database.sqlite")
cursor2 = conn2.cursor()

##**Consulta 1**

Se obtienen los nombres de las tablas y las columnas para cada una de estas.   

In [ ]:
# Su codigo aca
tablas = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';",conn2)
print(f"Esta base de datos contiene {len(tablas)} tablas: {', '.join(tablas['name'].values)}")

Esta base de datos contiene 8 tablas: sqlite_sequence, Player_Attributes, Player, Match, League, Country, Team, Team_Attributes


In [ ]:
for n in tablas["name"]:
  print("Nombre de Tabla: " + n)
  print("Columnas: ")
  display(pd.read_sql(f"PRAGMA table_info({n})",conn2))
  print("\n")

Nombre de Tabla: sqlite_sequence
Columnas: 


,cid,name,type,notnull,dflt_value,pk
0,0,name,,0,None,0
1,1,seq,,0,None,0




Nombre de Tabla: Player_Attributes
Columnas: 


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,player_fifa_api_id,INTEGER,0,None,0
2,2,player_api_id,INTEGER,0,None,0
3,3,date,TEXT,0,None,0
4,4,overall_rating,INTEGER,0,None,0
5,5,potential,INTEGER,0,None,0
6,6,preferred_foot,TEXT,0,None,0
7,7,attacking_work_rate,TEXT,0,None,0
8,8,defensive_work_rate,TEXT,0,None,0
9,9,crossing,INTEGER,0,None,0




Nombre de Tabla: Player
Columnas: 


...

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,name,TEXT,0,None,0




Nombre de Tabla: Team
Columnas: 


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,team_api_id,INTEGER,0,None,0
2,2,team_fifa_api_id,INTEGER,0,None,0
3,3,team_long_name,TEXT,0,None,0
4,4,team_short_name,TEXT,0,None,0




Nombre de Tabla: Team_Attributes
Columnas: 


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,team_fifa_api_id,INTEGER,0,None,0
2,2,team_api_id,INTEGER,0,None,0
3,3,date,TEXT,0,None,0
4,4,buildUpPlaySpeed,INTEGER,0,None,0
5,5,buildUpPlaySpeedClass,TEXT,0,None,0
6,6,buildUpPlayDribbling,INTEGER,0,None,0
7,7,buildUpPlayDribblingClass,TEXT,0,None,0
8,8,buildUpPlayPassing,INTEGER,0,None,0
9,9,buildUpPlayPassingClass,TEXT,0,None,0


Los nombres de las columnas no son auto explicativas, sobretodo para quienes no esten familiarizados con el mundo del fútbol.

Por ejemplo, la tabla `Player_Attibutes` se logra identificar que entrega información sobre distintas cualidades o atributos de cada jugador calificandolos con numeros enteros, pero no se indica qué significan estas cualidades ni los rangos de cada uno de estos atributos. Pasa algo similar con las tablas `Match` y `Team_Attributes`.

Por otro lado las tablas `Player` o `Country` son más sencillas y se puede identificar a que corresponde cada columna.

##**Consulta 2**

Se muestran los países existentes en la tabla Country, junto a su identificador.

In [ ]:
# Su codigo aca
pd.read_sql("""
              SELECT *
              FROM Country
            """,conn2)

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


##**Consulta 3**

Se consulta la cantidad de  partidos en la tabla Matches para los países Italia, Suiza y España y se muestran los primeros 20 registros de este match especificando las primeras 11 columnas de dicha tabla.

In [ ]:
# Su codigo aca
cantidad_partidos = pd.read_sql("""
              SELECT COUNT(*)
              FROM Match
              WHERE country_id IN (21518, 24558, 10257)
            """,conn2).values[0][0]
print(f"Hay {cantidad_partidos} partidos para Italia, Suiza y España")

Hay 7479 partidos para Italia, Suiza y España


In [ ]:
columnas = pd.read_sql("PRAGMA table_info(Match)",conn2)['name'][:11].to_list()
pd.read_sql("""
              SELECT *
              FROM Match
              WHERE country_id IN (21518, 24558, 10257)
            """,conn2)[columnas].head(20)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,10257,10257,10257,2008/2009,1,2008-08-31 00:00:00,537632,8524,8551,1,0
1,10258,10257,10257,2008/2009,1,2008-08-31 00:00:00,537633,8529,8543,1,4
2,10259,10257,10257,2008/2009,1,2008-08-31 00:00:00,537634,8530,10233,1,0
3,10260,10257,10257,2008/2009,1,2008-08-31 00:00:00,537635,8533,8690,2,1
4,10261,10257,10257,2008/2009,1,2008-08-31 00:00:00,537636,8535,9885,1,1
5,10262,10257,10257,2008/2009,1,2008-08-31 00:00:00,537637,8564,9857,1,2
6,10263,10257,10257,2008/2009,1,2008-08-31 00:00:00,537638,8686,9875,1,1
7,10264,10257,10257,2008/2009,1,2008-08-30 00:00:00,537639,9882,8636,1,1
8,10265,10257,10257,2008/2009,1,2008-08-31 00:00:00,537640,9804,9888,3,0
9,10266,10257,10257,2008/2009,1,2008-08-30 00:00:00,537641,8600,8540,3,1


##**Consulta 4**

Se consulta cuántos partidos hay en la tabla Matches con 5 goles mínimo para el
equipo de local o visita?

In [ ]:
# Su codigo aca
partidos_mas_5_goles = pd.read_sql("""
              SELECT COUNT(*)
              FROM Match
              WHERE  home_team_goal>=5 OR away_team_goal >= 5
            """,conn2).values[0][0]
print(f"Hay {partidos_mas_5_goles} partidos con 5 goles mínimo para el equipo de local o visita")

Hay 962 partidos con 5 goles mínimo para el equipo de local o visita


##**Consulta 5**

Se consulta por cuantos registros, valores mínimos, valores máximos, y promedios para las columnas overall score, shot_power, stamina, short_passing y
long_passing.

In [ ]:
# Su codigo aca
pd.read_sql("""
              SELECT  "overall_rating", "shot_power", "stamina", "short_passing", "long_passing"
              FROM Player_Attributes
            """,conn2).describe()

,overall_rating,shot_power,stamina,short_passing,long_passing
count,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000
mean,68.600015,61.808427,67.038544,62.429672,57.069880
std,7.041139,16.135143,13.165262,14.194068,14.394464
min,33.000000,2.000000,10.000000,3.000000,3.000000
25%,64.000000,54.000000,61.000000,57.000000,49.000000
50%,69.000000,65.000000,69.000000,65.000000,59.000000
75%,73.000000,73.000000,76.000000,72.000000,67.000000
max,94.000000,97.000000,96.000000,97.000000,97.000000


Ahora, se hace la misma consulta anterior pero filtrando por un overall_score entre 50 y 70. Con esto se dejan afuera cerca de 75 mil registros y los promedios de cada atributo disminuyen con respecto a la consulta si el filtro aplicado.

In [ ]:
pd.read_sql("""
              SELECT  "overall_rating", "shot_power", "stamina", "short_passing", "long_passing"
              FROM Player_Attributes
              WHERE "overall_rating" BETWEEN 50 AND 70
            """,conn2).describe()

,overall_rating,shot_power,stamina,short_passing,long_passing
count,108822.000000,108822.000000,108822.000000,108822.000000,108822.000000
mean,64.310571,57.646413,64.620490,58.438018,53.116383
std,4.478648,15.010983,12.615483,12.605880,12.942947
min,50.000000,3.000000,10.000000,3.000000,5.000000
25%,62.000000,50.000000,58.000000,54.000000,46.000000
50%,65.000000,61.000000,66.000000,62.000000,56.000000
75%,68.000000,68.000000,73.000000,67.000000,63.000000
max,70.000000,94.000000,94.000000,90.000000,86.000000


##**Consulta 6**

Se consulta cuantos partidos hay en la tabla Matches con 5 goles mínimo entre los dos equipos

In [ ]:
partidos_mas_5_goles_combinado = pd.read_sql("""
              SELECT COUNT(*)
              FROM Match
              WHERE  (home_team_goal+away_team_goal) >= 5
            """,conn2).values[0][0]
print(f"Hay {partidos_mas_5_goles_combinado} partidos con 5 goles mínimo entre los dos equipos")

Hay 3629 partidos con 5 goles mínimo entre los dos equipos


#**Fin Actividad 1**

#**Agrupamiento: GROUP BY, HAVING**

Anteriormente, cuando nosotros aplicábamos funciones de agregación sobre nuestras columnas/tablas, el motor de base de datos automáticamente genera un grupo (el cual indica que es toda la tabla). Esto se le conoce como los grupos **implícitos**, debido a que no existe una cláusula para detallar como generar estos grupos. Por otra parte, los grupos **Explícitos** se generan cuando nosotros damos la instrucción al motor de base de datos sobre como generar los grupos. Para esto, nosotros podemos utilizar la cláusula `GROUP BY` indicando cual columna nosotros queremos generar los grupos. La sintaxis para utilizar generar los grupos corresponde a la siguiente:

```SQL
SELECT column1, FUNC(column2), FUNC(column3), ..., FUNC(columnN) FROM Tabla GROUP BY column1
SELECT column1, FUNC(column2), FUNC(column3), ..., FUNC(columnN) FROM Tabla WHERE (condicion_logica) GROUP BY column1
```

Explorando con nuestros ejemplos: veamos qué pasa si hacemos los grupos, pero sin aplicar ninguna función de agregación:


In [ ]:
pd.read_sql("SELECT GenreId FROM Track GROUP BY GenreId",conn)

,GenreId
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


In [ ]:
pd.read_sql("SELECT DISTINCT GenreId FROM Track",conn)

,GenreId
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


Ahora, agruparemos y utlizaremos funciones de agregación para responder a la siguiente consulta: <br>
**Para cada estilo musical, cuántas canciones tiene?, cuál es la duración promedio de sus canciones?**

In [ ]:
pd.read_sql("""
              SELECT GenreId, COUNT(GenreId), AVG(Milliseconds)/(60*1000)
              FROM Track
              GROUP BY GenreId
            """,conn)

,GenreId,COUNT(GenreId),AVG(Milliseconds)/(60*1000)
0,1,1297,4.731834
1,2,130,4.862590
2,3,374,5.162491
3,4,332,3.905897
4,5,12,2.244058
5,6,81,4.505996
6,7,579,3.880988
7,8,58,4.119629
8,9,48,3.817235
9,10,43,4.072848


También podemos aplicar un filtro a través de la cláusula `WHERE`:

In [ ]:
pd.read_sql("""
              SELECT GenreId, COUNT(GenreId), AVG(Milliseconds)/(60*1000)
              FROM Track
              WHERE UnitPrice > 1.5
              GROUP BY GenreId
              """,conn)

,GenreId,COUNT(GenreId),AVG(Milliseconds)/(60*1000)
0,18,13,43.759151
1,19,93,35.750684
2,20,26,48.529717
3,21,64,42.921396
4,22,17,26.421062


Cabe destacar que el filtro de WHERE se realiza previamente a realizar los grupos. Sin embargo, también es posible filtrar después de realizar los grupos utilizando la cláusula `HAVING` y cuya sintaxis es:

```SQL
SELECT column1, FUNC(column2), FUNC(column3), ..., FUNC(columnN) FROM Tabla GROUP BY column1 HAVING (condicion_logica1)
SELECT column1, FUNC(column2), FUNC(column3), ..., FUNC(columnN) FROM Tabla WHERE (condicion_logica1) GROUP BY column1 HAVING (condicion_logica2)
```

Viendo un ejemplo utilizando nuestras tablas:


In [ ]:
pd.read_sql("""
              SELECT GenreId, COUNT(GenreId), AVG(Milliseconds)/(60*1000)
              FROM Track
              GROUP BY GenreId
              HAVING Milliseconds BETWEEN 100000 AND 300000
            """, conn)

,GenreId,COUNT(GenreId),AVG(Milliseconds)/(60*1000)
0,2,130,4.862590
1,3,374,5.162491
2,4,332,3.905897
3,5,12,2.244058
4,6,81,4.505996
5,7,579,3.880988
6,8,58,4.119629
7,9,48,3.817235
8,10,43,4.072848
9,12,24,3.152737


Con cláusula HAVING nuestras condiciones lógicas pueden incluir agregaciones dependientes de los grupos:

In [ ]:
pd.read_sql("""
              SELECT GenreId, COUNT(GenreId), AVG(Milliseconds)/(60*1000)
              FROM Track
              GROUP BY GenreId
              HAVING COUNT() < 50
            """, conn)

,GenreId,COUNT(GenreId),AVG(Milliseconds)/(60*1000)
0,5,12,2.244058
1,9,48,3.817235
2,10,43,4.072848
3,11,15,3.659833
4,12,24,3.152737
5,13,28,4.957549
6,15,30,5.049763
7,16,28,3.748730
8,17,35,2.969605
9,18,13,43.759151


Por otra parte, si ejecutamos esta misma consulta, pero en vez de utilizar la cláusula `HAVING` ponemos la agregación en una cláusula `WHERE`, el código debería generar un error:

In [ ]:
pd.read_sql("""
              SELECT GenreId, COUNT(GenreId), AVG(Milliseconds)/(60*1000)
              FROM Track
              WHERE COUNT(*) < 50
              GROUP BY GenreId
            """, conn)

DatabaseError: Execution failed on sql '
              SELECT GenreId, COUNT(GenreId), AVG(Milliseconds)/(60*1000)
              FROM Track
              WHERE COUNT(*) < 50
              GROUP BY GenreId
            ': misuse of aggregate: COUNT()

#**Ordenar los valores retornados de una consulta**

Para poder ordenar los valores retornados de una consulta desde SQL podemos usar la sentencia `ORDER BY`, la cual tiene que ir al final de la consulta SQL. La sintaxis corresponde a la siguiente:

```SQL
SELECT column1, column2, column3, ..., columnN FROM Tabla ORDER BY column2
SELECT column1, column2, column3, ..., columnN FROM Tabla ORDER BY column2 DESC

SELECT column1, column2, column3, ..., columnN FROM Tabla ORDER BY 2
SELECT column1, column2, column3, ..., columnN FROM Tabla ORDER BY 2 DESC
```

En este caso, podemos ver dos cosas interesantes, ya sea podemos ordenar por el nombre de la columna obtenida en la respuesta, o en su defecto según su índice (column2, en este caso corresponde al índice 2) partiendo desde el valor 1. Lo segundo que podemos observar, corresponde a incluir o no la cláusula `DESC`. Por defecto, `ORDER BY` ordena de forma **ascendente**, y utilizando `DESC` obtenemos la forma **descendente**. Veamos unos ejemplos extendiendo lo visto con `GROUP BY` y `HAVING`


Ordenaremos usando el tercer atributo, desde izquierda a derecha, de la cláusula `SELECT`:

In [ ]:
pd.read_sql("""
              SELECT GenreId, COUNT(GenreId), AVG(Milliseconds)/(60*1000)
              FROM Track
              GROUP BY GenreId
              HAVING COUNT(*) < 50
              ORDER BY 3
            """, conn)

,GenreId,COUNT(GenreId),AVG(Milliseconds)/(60*1000)
0,5,12,2.244058
1,25,1,2.913550
2,17,35,2.969605
3,12,24,3.152737
4,11,15,3.659833
5,16,28,3.748730
6,9,48,3.817235
7,10,43,4.072848
8,23,40,4.400975
9,13,28,4.957549


Si renombramos el atributo podemos incluir su nuevo nombre en `ORDER BY`:

In [ ]:
pd.read_sql("""
              SELECT GenreId, COUNT(GenreId), AVG(Milliseconds)/(60*1000) as Promedio
              FROM Track
              GROUP BY GenreId
              HAVING COUNT(*) < 50
              ORDER BY Promedio DESC
            """,conn)

,GenreId,COUNT(GenreId),Promedio
0,20,26,48.529717
1,18,13,43.759151
2,22,17,26.421062
3,15,30,5.049763
4,13,28,4.957549
5,23,40,4.400975
6,10,43,4.072848
7,9,48,3.817235
8,16,28,3.748730
9,11,15,3.659833


#**Actividad 2**

Utilizando el mismo set de datos de la **Actividad 1**, vamos a hacer prácticas utilizando todo lo aprendido incluyendo agregaciones, grupos y condiciones lógicas. Para esto, ustedes tienen que escribir las consultas SQL para responder las siguientes preguntas:

1. ¿Cuantos partidos hay por país en la tabla Matches?
2. ¿Cuantos partidos hay por liga, promedio de goles de equipo visitante y promedio de goles de equipo local en la tabla Matches, para las temporadas 2010/2011, 2012/2013 y 2014/2015?
3. Obtenga el identificador, los valores promedio mínimo y máximos de las columnas defencePressure, defenseAggression, chanceCreationPassing, chanceCreationShooting de la tabla Team Attributes (Agrupar por **team_api_id** o **team_fifa_api_id**). Reporte las 10 primeras filas mostrando el resultado de esta query.
4. Reporte el nombre, el overall_rating **promedio** y la fecha de nacimiento para los jugadores con una cantidad de registros mayor que 40. Para esto consulte en primer lugar la Tabla **Player_Attributes** para obtener la agrupación por overall_rating y realizar el filtrado. Una vez realizado esto, obtenga los nombres y cumpleaños desde la tabla Player. ¿Reconoce a algún jugador chileno? **Nota:** Cabe destacar que existe la columna **id** detallando el identificador de registro y **player_api_id**, detallando el identificador del jugador.
5. Reporte el nombre, la resta entre el overall_rating máximo y mínimo para los jugadores con una cantidad de registros mayor que 40. Ordene estos resultados en forma descendente.

##**Consulta 1**

Se consulta cuántos partidos hay por país en la tabla Matches

In [ ]:
# Su codigo aca
query = """
              SELECT country_id, COUNT(country_id) as CantidadPartidos
              FROM Match
              GROUP BY country_id
            """
pd.read_sql(query,conn2)

,country_id,CantidadPartidos
0,1,1728
1,1729,3040
2,4769,3040
3,7809,2448
4,10257,3017
5,13274,2448
6,15722,1920
7,17642,2052
8,19694,1824
9,21518,3040


##**Consulta 2**

Se consultan cuantos partidos hay por liga, promedio de goles de equipo visitante y promedio de goles de equipo local en la tabla Matches, para las temporadas 2010/2011, 2012/2013 y 2014/2015

In [ ]:
# Su codigo aca
query = """
              SELECT league_id, COUNT(league_id) as CantidadPartidos, AVG(away_team_goal) as PromedioGolesVisita, AVG(home_team_goal) as PromedioGolesLocal
              FROM Match
              WHERE season IN ('2010/2011', '2012/2013', '2014/2015')
              GROUP BY league_id
            """
pd.read_sql(query,conn2)

,league_id,CantidadPartidos,PromedioGolesVisita,PromedioGolesLocal
0,1,720,1.212500,1.573611
1,1729,1140,1.168421,1.551754
2,4769,1140,1.052632,1.407018
3,7809,918,1.261438,1.608932
4,10257,1139,1.137840,1.474978
5,13274,918,1.350763,1.800654
6,15722,720,1.051389,1.454167
7,17642,786,1.137405,1.424936
8,19694,684,1.187135,1.435673
9,21518,1140,1.135965,1.620175


##**Consulta 3**

Para obtener lo pedido primero visualizamos como es la tabla a consultar

In [ ]:
Team_Attributes = pd.read_sql("SELECT * FROM Team_Attributes",conn2)
Team_Attributes.head()

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover


Luego hacemos la consulta, limintando los resultados a mostrar en 10.

In [ ]:
# Su codigo aca
query = """
              SELECT team_api_id,
                    AVG(defencePressure), MIN(defencePressure), MAX(defencePressure),
                    AVG(defenceAggression), MIN(defenceAggression), MAX(defenceAggression),
                    AVG(chanceCreationPassing), MIN(chanceCreationPassing), MAX(chanceCreationPassing),
                    AVG(chanceCreationShooting), MIN(chanceCreationShooting), MAX(chanceCreationShooting)
              FROM Team_Attributes

              GROUP BY team_api_id
              LIMIT 10
            """
pd.read_sql(query,conn2)

,team_api_id,AVG(defencePressure),MIN(defencePressure),MAX(defencePressure),AVG(defenceAggression),MIN(defenceAggression),MAX(defenceAggression),AVG(chanceCreationPassing),MIN(chanceCreationPassing),MAX(chanceCreationPassing),AVG(chanceCreationShooting),MIN(chanceCreationShooting),MAX(chanceCreationShooting)
0,1601,47.166667,43,65,47.333333,44,60,55.500000,44,68,53.500000,50,70
1,1773,43.000000,43,43,44.000000,44,44,50.000000,50,50,54.000000,54,54
2,1957,49.333333,32,70,56.333333,50,70,50.833333,45,53,67.000000,64,70
3,2033,42.200000,37,50,33.800000,24,45,51.800000,50,53,48.200000,43,65
4,2182,51.666667,47,60,48.500000,37,58,56.666667,45,68,68.000000,67,70
5,2183,49.500000,31,57,40.750000,32,65,48.000000,39,69,59.250000,40,74
6,2186,59.000000,48,70,44.400000,33,70,55.400000,40,66,52.200000,33,78
7,4087,46.600000,45,47,53.200000,43,56,51.600000,50,52,58.400000,48,65
8,4170,50.000000,35,70,53.000000,44,60,39.000000,35,47,58.333333,50,65
9,6269,41.750000,35,60,48.500000,45,51,53.000000,50,56,54.250000,49,61


##**Consulta 4**

Reporte el nombre, el overall_rating promedio y la fecha de nacimiento para los jugadores con una cantidad de registros mayor que 40. Para esto consulte en primer lugar la Tabla Player_Attributes para obtener la agrupación por overall_rating y realizar el filtrado.

In [ ]:
Player_Attributes = pd.read_sql("SELECT * FROM Player_Attributes",conn2)
Player_Attributes.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [ ]:
# Su codigo aca
query = """
              SELECT player_api_id, COUNT(player_api_id) as CantidadRegistros, AVG(overall_rating) as PromedioOverall
              FROM Player_Attributes
              GROUP BY player_api_id
              HAVING CantidadRegistros > 40

            """
pd.read_sql(query,conn2)

,player_api_id,CantidadRegistros,PromedioOverall
0,7284,41,67.658537
1,11685,43,79.395349
2,21446,41,81.439024
3,22834,43,72.790698
4,23019,41,67.487179
...,...,...,...
98,292313,41,72.439024
99,355358,41,66.926829
100,363333,44,61.886364
101,364520,41,75.097561


Luego se obtienen los nombres y cumpleaños desde la tabla Player de los jugadores que tienen una cantidad de registros mayor a 40. Se reconocen los futbolistas chilenos Arturo Vidal, Gary Medel y Mauricio Isla.



In [ ]:
query = """
              SELECT player_name, birthday
              FROM Player
              WHERE player_api_id IN (SELECT player_api_id
                                      FROM Player_Attributes
                                      GROUP BY player_api_id
                                      HAVING COUNT(player_api_id) > 40)
            """
pd.read_sql(query,conn2)

,player_name,birthday
0,Kasper Haemaelaeinen,1986-08-08 00:00:00
1,Antonio Candreva,1987-02-28 00:00:00
2,Hernanes,1985-05-29 00:00:00
3,Marek Suchy,1988-03-29 00:00:00
4,Giles Barnes,1988-08-05 00:00:00
...,...,...
98,Raul Jimenez,1991-05-05 00:00:00
99,Christian Guenter,1993-02-28 00:00:00
100,Dele Alli,1996-04-11 00:00:00
101,Domenico Berardi,1994-08-01 00:00:00


In [ ]:
pd.read_sql(query,conn2)["player_name"].to_list()

['Kasper Haemaelaeinen',
 'Antonio Candreva',
 'Hernanes',
 'Marek Suchy',
 'Giles Barnes',
 'Greg Halford',
 'Leonardo Bonucci',
 'Riccardo Montolivo',
 'Nigel Reo-Coker',
 'Cristian Zapata',
 'Andrea Ranocchia',
 'Goran Pandev',
 'Zdravko Kuzmanovic',
 'Michael Bradley',
 'Hamit Altintop',
 'Jermaine Jones',
 'Alberto Aquilani',
 'Antonio Di Natale',
 'Phillippe Mexes',
 'Stephan Lichtsteiner',
 'Thierry Henry',
 'Landon Donovan',
 'Andrea Pirlo',
 'Markus Rosenberg',
 'Tim Cahill',
 'Andrea Barzagli',
 'Rodrigo Palacio',
 'Christopher Maguire',
 'Fernando Llorente',
 'Umut Bulut',
 'Fin Bartels',
 'Clemens Fritz',
 'Mahamadou Samassa',
 'Obafemi Martins',
 'Gervinho',
 'Laurent Ciman',
 'Henok Goitom',
 'Kari Arnason',
 'Fabio Quagliarella',
 'Adrien Silva',
 'Nani',
 'Kaka',
 'Fredy Guarin',
 'Alessio Cerci',
 'Radja Nainggolan',
 'Federico Peluso',
 'Giorgio Chiellini',
 'Claudio Marchisio',
 'Sebastian Meoli',
 'Simone Padoin',
 'Guillermo Molins',
 'Christian Fuchs',
 'Bakary Sa

##**Consulta 5**

Reporte el nombre, la resta entre el overall_rating máximo y mínimo para los jugadores con una cantidad de registros mayor que 40. Ordene estos resultados en forma descendente.

In [ ]:
Player_Attributes = pd.read_sql("SELECT * FROM Player_Attributes",conn2)
Player_Attributes.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [ ]:
player = pd.read_sql("SELECT * FROM Player",conn2)
player.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [ ]:
# Su codigo aca
query = """
              SELECT p.player_name, MAX(pa.overall_rating) - MIN(pa.overall_rating) as RangoOverall
              FROM Player p INNER JOIN Player_Attributes pa
              ON p.player_api_id = pa.player_api_id
              GROUP BY p.player_api_id
              HAVING COUNT(pa.player_api_id) > 40
              ORDER BY RangoOverall DESC
            """
pd.read_sql(query,conn2)

,player_name,RangoOverall
0,Marco Reus,42
1,Bakary Sako,38
2,Radja Nainggolan,37
3,Leonardo Bonucci,36
4,Yacine Brahimi,34
...,...,...
98,Stephan Lichtsteiner,7
99,Mario Balotelli,6
100,Riku Riski,6
101,Rodrigo Palacio,6


#**Fin Actividad 2**

# <font color='red'>__LINK DE INTERÉS__: Uso de `GROUP BY` y `HAVING`</font>

El uso de las sentencias `GROUP BY` y `HAVING` lo podemos ver en el siguiente link [aquí](https://www.datacamp.com/es/tutorial/group-by-having-clause-sql).

# <font color='purple'> __EXPERIMENTO__: </font>
### Consulta y comparación de atributos de jugadores según pie de preferencia (zurdos o diestros).

En el siguiente experimiento, se consultan algunos atributos de jugadores y se calcula el promedio de ellos segun si los jugadores son zurdos o diestros.

In [ ]:
query = """
              SELECT pa.preferred_foot,
              COUNT(pa.preferred_foot),
              AVG(pa.overall_rating),
              AVG(pa.potential),
              AVG(pa.crossing),
              AVG(pa.finishing),
              AVG(pa.heading_accuracy),
              AVG(pa.short_passing),
              AVG(pa.long_passing),
              AVG(pa.ball_control)
              FROM Player p INNER JOIN Player_Attributes pa
              ON p.player_api_id = pa.player_api_id
              GROUP BY pa.preferred_foot
              """
pd.read_sql(query,conn2)

,preferred_foot,COUNT(pa.preferred_foot),AVG(pa.overall_rating),AVG(pa.potential),AVG(pa.crossing),AVG(pa.finishing),AVG(pa.heading_accuracy),AVG(pa.short_passing),AVG(pa.long_passing),AVG(pa.ball_control)
0,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,left,44733,68.626182,73.506673,60.391054,49.533767,57.430555,64.291351,59.455525,65.416941
2,right,138409,68.591558,73.445383,53.372606,50.046254,57.212847,61.827988,56.298853,62.733421


Se observa que los jugadores zurdos tienen en promedio mejores pases cortos, pases largos y mejor dominio del balon que los jugadores diestros.

## <font color='purple'>Fin experimento </font>